In [136]:
import pandas as pd
import numpy as np
import datetime
import math
from copy import deepcopy

Path = "G:/.shortcut-targets-by-id/1sZS2kL5L66960IYJ4xfNEV9CuKrs_7fE/1조/RAW DATA"

In [2]:
# df = pd.read_csv(Path+"/월별기상관측(2000.01-2021.12).csv",encoding="cp949")
# df.insert(1,'행정구역',object)

# df.to_csv(Path+"/수정_월별기상관측.csv")

In [3]:
# df = pd.read_csv(Path+"/수정_월별기상관측.csv")
# # df.head()

# # 기상 측정소 위치 라벨링
# df.loc[df['지점명']=='서울', '행정구역'] = '서울특별시'
# df.loc[df['지점명']=='무안', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='진도(첨찰산)', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='대구(기)', '행정구역'] = '대구광역시'
# df.loc[df['지점명']=='주암', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='성산포', '행정구역'] = '제주도'

In [4]:
# df_new[df_new["행정구역"]=="<class 'object'>"]["지점명"].unique()
# df.loc[df['지점명']=='주암']
# df.to_csv(Path+"/최종_월별기상관측.csv",encoding="cp949")

In [5]:
#1. 최대일사량 결측치 처리: 동일년도+동일월+동일 행정구역 데이터의 중앙값으로 대체 ex)강원도의 철원지역 결측치 2021년 1월 강원도의 최대일사량
#    (1) 세종시 : 기간 2000년 01월~2019년 04월 충청북도 최대일사량 중앙값으로 대체
 

In [6]:
# df = pd.read_csv(Path+"/최종_월별기상관측.csv",encoding="cp949")

In [7]:
# df.drop(df.columns[:2],axis=1,inplace=True)   # 0,1번째 컬럼 drop
# # df.head()

In [8]:
# 필요 컬럼만 추출('행정구역', "지점명", '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)')
# df_new=df[['행정구역', "지점명", '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]

In [9]:
# df_ulsan = df_new[df_new["행정구역"]=="울산광역시"][["행정구역", "합계 일사량(MJ/m2)"]]

# print(len(df_ulsan))
# print(df_ulsan["합계 일사량(MJ/m2)"].isnull().sum())

# 울산광역시의 일사량은 전부 NaN 값

In [10]:
# df_new["합계 일사량(MJ/m2)"].replace(0.0, np.nan, inplace=True)   # 일사량 0.0을 전부 NaN으로 변경

In [11]:
# df_new[df_new["합계 일사량(MJ/m2)"]==0.0]["합계 일사량(MJ/m2)"]

In [12]:
# 일시와 행정구역에 따른 합계 일사량 중앙값
# sunshine = pd.pivot_table(df_new, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [13]:
# sunshine.interpolate(inplace=True)  # 부산광역시 18년도 8월 결측치를 앞뒤 값으로 보간

In [14]:
# df2 = df_new

In [15]:
# df["합계 일사량(MJ/m2)"].fillna()
# for i in range(len(df2)) : 
#     dt = df2.loc[i,"일시"]
#     dstr = df2.loc[i,"행정구역"]
# #     print(dt)
# #     print(dstr)
#     if np.isnan(df2.loc[i,"합계 일사량(MJ/m2)"]) :  # 일사량이 NaN일 때
#         if df2.loc[i,"행정구역"] == "세종특별자치시" :       # 행정구역이 세종시라면
#             df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine["충청북도"][dt]     # 동일 일시의 충청북도 중앙값을 채움
#         elif df2.loc[i,"행정구역"] == "울산광역시" :        # 행정구역이 울산광역시라면
#             df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine["경상남도"][dt]     # 동일 일시의 경상남도 중앙값을 채움
#         else : 
#             df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine[dstr][dt]   # 동일 일시, 동일 행정 구역의 중앙값을 채움

# # pd.pivot_table(df2, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [16]:
# df2[df2["행정구역"]=="세종특별자치시"][["일시","합계 일사량(MJ/m2)"]]

In [17]:
# df2[df2["행정구역"]=="충청북도"][["일시","합계 일사량(MJ/m2)"]]

In [18]:
# df2[df2["합계 일사량(MJ/m2)"].isnull()]
# ["울산광역시", "세종특별자치시"]

In [19]:
# avr_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="평균기온(°C)",aggfunc="median")

In [20]:
# high_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="최고기온(°C)",aggfunc="median")

In [21]:
# low_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="최저기온(°C)",aggfunc="median")

In [22]:
# df2.info()

In [23]:
# def fill_tem(temperature, dataframe, pivot) : 
#     for i in range(len(dataframe)) : 
#         dt = dataframe.loc[i,"일시"]
#         dstr = dataframe.loc[i,"행정구역"]

#         if np.isnan(dataframe.loc[i,temperature]) :  # 기온 관련 컬럼값이 NaN일 때 
#             dataframe.loc[i,temperature] = pivot[dstr][dt]   # 동일 일시, 동일 행정 구역의 중앙값을 채움
    
#     return dataframe

In [24]:
# df2 = fill_tem("최저기온(°C)", df2, low_tem)
# df2 = fill_tem("최고기온(°C)", df2, high_tem)
# df2 = fill_tem("평균기온(°C)", df2, avr_tem)

In [25]:
# df2_tst[df2_tst["평균기온(°C)"].isnull()]

In [26]:
# df2['평균기온(°C)'] = df2['평균기온(°C)'].fillna(value=avr_tem["충청북도"]["2019-05"])

In [27]:
# df2.info()

In [28]:
# df2[df2["행정구역"]=="세종특별자치시"][["일시","합계 일사량(MJ/m2)"]]

In [29]:
# df2[df2["행정구역"]=="충청북도"][["일시","합계 일사량(MJ/m2)"]]

In [30]:
# df2_test=pd.pivot_table(df2, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [31]:
# df2_test

In [32]:
# chch_copy = df2[df2['행정구역']=='충청북도']
# chch_copy

In [33]:
# chch_median = chch_copy.groupby("일시").median()

In [34]:
# chch_median.insert(0,column="행정구역", value="세종특별자치시")
# chch_median.insert(1,column="지점명", value="세종")
# # chch_median

In [35]:
# chch_median.reset_index(inplace=True)
# chch_median

In [36]:
# chch_median.insert(3,column="일시",value=chch_median["일시"])
# chch_median
# chch_median.columns

In [37]:
# chch_median = chch_median[['행정구역', '지점명', '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]
# chch_median

In [38]:
# idx_chch = chch_median[chch_median["일시"]=="2019-05"].index[0]
# idx_chch

In [39]:
# chch_bef_201905 = chch_median.iloc[:idx_chch,:]
# # chch_bef_201905

In [40]:
# df2[df2["행정구역"]=="세종특별자치시"]

In [41]:
# new_df = pd.concat([df2.iloc[:14984],chch_bef_201905,df2.iloc[14984:]])
# # new_df

In [42]:
# new_df[new_df["행정구역"]=="세종특별자치시"]

In [43]:
# new_df.info()

In [44]:
# new_df[new_df['행정구역']=='세종특별자치시'].isnull().sum()

In [45]:
# new_df.fillna(0.0,inplace=True)
# new_df.info()

In [46]:
# new_df.to_csv("data/최종_최종__월별기상관측.csv",encoding="cp949")

In [47]:
# new_df.groupby(["행정구역", "일시"]).median()

In [48]:
new_df = pd.read_csv("./data/최종_최종__월별기상관측.csv", encoding="cp949")
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23078 entries, 0 to 23077
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          23078 non-null  int64  
 1   행정구역                23078 non-null  object 
 2   지점명                 23078 non-null  object 
 3   일시                  23078 non-null  object 
 4   평균기온(°C)            23078 non-null  float64
 5   최고기온(°C)            23078 non-null  float64
 6   최저기온(°C)            23078 non-null  float64
 7   월합강수량(00~24h만)(mm)  23078 non-null  float64
 8   합계 일사량(MJ/m2)       23078 non-null  float64
dtypes: float64(5), int64(1), object(3)
memory usage: 1.6+ MB


In [49]:
new_df.drop("Unnamed: 0", axis=1, inplace=True)  # 0번째 컬럼 drop
# new_df.head(3)

In [50]:
new_df["일시"] = pd.to_datetime(new_df["일시"])  # 일시를 datetime으로 변경
# new_df.head()

In [51]:
new_df["연도"] =  new_df["일시"].dt.year   # 연도만 추출
new_df["월"] =  new_df["일시"].dt.month    # 월만 추출
# new_df.head(10)

In [52]:
# new_df.columns

In [53]:
# 컬럼 순서 변경
new_df = new_df[['행정구역', '지점명', '연도', '월', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]
# new_df.head(3)


In [54]:
# test = new_df.groupby(["연도","행정구역","월"]).median()[:24]

In [55]:
# test

In [56]:
# test2 = test.iloc[[0, 12]]
# for i in range(1, int(len(test)/2)) : 
#     a = test.iloc[[i,i+12]]
#     # print(a)
#     test2 = pd.merge(left = test2, right = a, left_index = True, right_index = True, how = "outer")
# test2

In [57]:
# test2 = test.iloc[0]
# for i in range(1, len(test)) : 
#     a = test.iloc[i]
#     test2 = pd.merge(left = test2, right = a, left_index = True, right_index = True, how = "outer") 
# test2

In [58]:
# i = 0, j = 0, k = 0, new_col = 0 empty_frame.iloc[0,0] = test.iloc[0,0]
#               k = 1, new_col = 1 empty_frame.iloc[0,1] = test.iloc[0,1]
#               k = 2, new_col = 2 empty_frame.iloc[0,2] = test.iloc[0,2]
#               k = 3, new_col = 3 empty_frame.iloc[0,3] = test.iloc[0,3]
#               k = 4, new_col = 4 empty_frame.iloc[0,4] = test.iloc[0,4]
# i = 1, j = 0, k = 0, new_col = 5 empty_frame.iloc[0,5] = test.iloc[1,0]
#               k = 1, new_col = 6 empty_frame.iloc[0,6] = test.iloc[1,1]
#               k = 2, new_col = 7 empty_frame.iloc[0,7] = test.iloc[1,2]
#               k = 3, new_col = 8 empty_frame.iloc[0,8] = test.iloc[1,3]
#               k = 4, new_col = 9 empty_frame.iloc[0,9] = test.iloc[1,4]
# i = 12, j = 1, k = 0, new_col = 60 empty_frame.iloc[0,5] = test.iloc[1,0]
#               k = 1, new_col = 6 empty_frame.iloc[0,6] = test.iloc[1,1]
#               k = 2, new_col = 7 empty_frame.iloc[0,7] = test.iloc[1,2]
#               k = 3, new_col = 8 empty_frame.iloc[0,8] = test.iloc[1,3]
#               k = 4, new_col = 9 empty_frame.iloc[0,9] = test.iloc[1,4]

In [59]:
real_test = new_df.groupby(["연도","행정구역","월"]).median() # 연도, 행정구역, 월별로 group화

In [60]:
empty_array = np.empty((int(len(real_test)/12),60))   # (4488/12, 60)의 구조를 가지는 빈 행렬을 생성
empty_array[:] = np.nan
empty_frame = pd.DataFrame(empty_array)

cname = []
for i in range(1,13) : 
    L = ['평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']
    for elem in L : 
        word = elem + "_{}월".format(i)  # 각 컬럼명에 1월부터 12월까지 붙여줌
        cname.append(word)

empty_frame.columns = cname   # 변수에 월수가 붙은 리스트를 컬럼명으로 대체

In [61]:
# cname

In [138]:
for i in range(len(real_test)) : 
    j = math.floor(i/12)  # 각 행 인덱스를 12로 나눈 후 내림  >>>  0부터 11까지는 0으로, 12부터 23까지는 1로.....
    remain = i%12  # 각 행 인덱스를 12로 나눈 나머지  >>> 0과 12, 24는 0으로, 1과 13 25는 1로......
    for k in range(len(real_test.iloc[i])) : 
        new_col = (remain*5)+k      # remain이 0이면 new_col은 0부터 4까지, remain이 1이면 new_col은 5부터 9까지, remain이 11이면 new_col은 55부터 59까지, 
        empty_frame.iloc[j,new_col] = real_test.iloc[i,k]  # real_test의 데이터를 empty_frame에 삽입
wanna_frame = empty_frame

In [139]:
b = real_test.index   # groupby한 데이터 프레임의 인덱스를 저장
a = [(b[i][0],b[i][1]) for i in range(len(b)) if i%12==0]   # 인덱스를 12로 나눈 후 나머지가 없는 행만 추출(연도는 같고 월만 다르기 때문에 그 중 처음만 가져온다)
                                                            # 그 후 연도와 행정구역만 튜플로 저장

In [64]:
# len(a)

In [140]:
new_index = pd.MultiIndex.from_tuples(a)  # 연도와 행정구역을 추출한 튜플 리스트를 멀티인덱스로 변환

In [141]:
wanna_frame.index = new_index
# wanna_frame.head()

In [67]:
admin_dist = np.sort(new_df["행정구역"].unique())

In [68]:
# city 배열 생성
city = ["서울특별시", "부산광역시", "대구광역시", "인천광역시", "광주광역시", "대전광역시", "울산광역시", "세종특별자치시", "경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "제주도"]
city_list= ["seoul", "busan", "daegu", "incheon", "gwangju", "daejeon", "ulsan", "sejong", "gyeonggi", "gangwon", "chungbuk", "chungnam", "jeonbuk", "jeonnam", "gyeongbuk", "gyeongnam", "jeju"]

# season 배열 생성
season = ["spring", "summer", "fall", "winter"]

# .query()를 사용하여 계절별로 데이터 프레임 생성 

# 지역별, 계절별 배추 생산량 csv 불러오기
for i in range(len(city)):
    for j in range(len(season)):
        globals()[city_list[i] + "_" + season[j]] = pd.read_csv("./data/"+city_list[i]+"_"+season[j]+".csv")
        # globals()[city_list[i] + "_" + season[j]] = new_df[(new_df["행정구역"]==city[i]) & (new_df["일시"].isin(season_list[j]))]
        # globals()[city_list[i] + "_" + season[j]].reset_index(drop=True, inplace=True)
        # globals()[city_list[i] + "_" + season[j]] = globals()[city_list[i] + "_" + season[j]].drop(["행정구역"], axis=1)
        # globals()[city_list[i] + "_" + season[j]] = globals()[city_list[i] + "_" + season[j]].rename(columns={"지점명": "지역명"})
        # # 지역명 컬럼 삭제
        # globals()[city_list[i] + "_" + season[j]] = globals()[city_list[i] + "_" + season[j]].drop(["지역명"], axis=1)
        # # 일시 데이터를 년도로 변경
        # globals()[city_list[i] + "_" + season[j]]["일시"] = globals()[city_list[i] + "_" + season[j]]["일시"].str[:4]
        # # csv 파일로 저장
        # globals()[city_list[i] + "_" + season[j]].to_csv("data/weather/" + city_list[i] + "_" + season[j] + ".csv", encoding="utf-8", index=False)

In [137]:
# seoul_fall_select

In [70]:
for i in range(len(city)):
    for j in range(len(season)):
        k_list = []
        for k in range(len(globals()[city_list[i] + "_" + season[j]].iloc[0])) : 
            if k % 3 == 2 or k % 3 == 0 : 
                k_list.append(k)
        
        globals()[city_list[i] + "_" + season[j]+ "_select"] = globals()[city_list[i] + "_" + season[j]].iloc[:,k_list]

In [71]:
year_list_spring = []
for i in range(2000,2022) : 
    year_list_spring.append((i,"일반봄배추:면적 (ha)"))
    for k in range(i,i+1) : 
        year_list_spring.append((k,"생산량 (톤)"))

In [72]:
year_list_summer = []
for i in range(2000,2022) : 
    year_list_summer.append((i,"고랭지배추:면적 (ha)"))
    for k in range(i,i+1) : 
        year_list_summer.append((k,"생산량 (톤)"))

In [73]:
year_list_fall = []
for i in range(2000,2022) : 
    year_list_fall.append((i,"노지가을배추:면적 (ha)"))
    for k in range(i,i+1) : 
        year_list_fall.append((k,"생산량 (톤)"))
# year_list

In [83]:
year_list_winter = []
for i in range(2014,2022) : 
    year_list_winter.append((i,"노지겨울배추:면적 (ha)"))
    for k in range(i,i+1) : 
        year_list_winter.append((k,"생산량 (톤)"))

In [75]:
# year_list = [(i,i) for i in range(2000,2022)]
# year_list

In [84]:
year_list_spring = pd.MultiIndex.from_tuples(year_list_spring)
year_list_summer = pd.MultiIndex.from_tuples(year_list_summer)
year_list_fall = pd.MultiIndex.from_tuples(year_list_fall)
year_list_winter = pd.MultiIndex.from_tuples(year_list_winter)
# year_list

In [77]:
# for i in range(len(city)):
#     for j in range(len(season)):
#         print(len(globals()[city_list[i] + "_" + season[j]+ "_select"].iloc[0]))

In [85]:
for i in range(len(city)):
    for j in range(len(season)):
        # if city[i] == "세종특별자치시"
        globals()[city_list[i] + "_" + season[j]+ "_select"] = globals()[city_list[i] + "_" + season[j]+ "_select"].transpose()
        globals()[city_list[i] + "_" + season[j]+ "_select"].index = globals()["year_list_"+season[j]]
        # globals()[city_list[i] + "_" + season[j]+ "_select"] = globals()[city_list[i] + "_" + season[j]+ "_select"].reindex(globals()["year_list_"+season[j]])
        # 겨울은 16개다


In [98]:
# jeonnam_winter_select.drop([0],axis=1).unstack().droplevel(0, axis=1)

,노지겨울배추:면적 (ha),생산량 (톤)
2014,4022,280454
2015,3382,255442
2016,3181,241533
2017,4145,332042
2018,4684,354672
2019,3307,223797
2020,4167,276415
2021,3236,224283


In [99]:
for i in range(len(city)):
    for j in range(len(season)):
        globals()[city_list[i] + "_" + season[j]+ "_select"] = globals()[city_list[i] + "_" + season[j]+ "_select"].drop([0],axis=1).unstack().droplevel(0, axis=1)

In [111]:
for i in range(len(season)) : 
    wan_idx = wanna_frame.index  
    globals()["idx_"+season[i]] = [(wan_idx[j][0],season[i],wan_idx[j][1]) for j in range(len(wan_idx))]
    globals()["idx_"+season[i]] = pd.MultiIndex.from_tuples(globals()["idx_"+season[i]])

In [143]:
for i in range(len(season)) : 
    globals()["wanna_frame_"+season[i]] = deepcopy(wanna_frame)
    globals()["wanna_frame_"+season[i]].index = globals()["idx_"+season[i]]

In [134]:
# wanna_frame_spring = wanna_frame
# wanna_frame_spring.index = idx_spring
# wanna_frame_summer = wanna_frame
# wanna_frame_summer.index = idx_summer
# wanna_frame_fall = wanna_frame
# wanna_frame_fall.index = idx_fall
# wanna_frame_winter = wanna_frame
# wanna_frame_winter.index = idx_winter

In [146]:
wanna_frame_spring

평균기온(°C)_1월  최고기온(°C)_1월  최저기온(°C)_1월  \
2000 spring 강원도          -2.90         8.50       -16.00   
            경기도          -2.65        10.20       -15.00   
            경상남도          1.40        13.20        -9.60   
            경상북도         -0.30        10.70       -12.60   
            광주광역시         1.00        15.30        -8.80   
...                        ...          ...          ...   
2021 spring 전라남도          1.70        16.15       -13.90   
            전라북도         -0.40        14.30       -18.60   
            제주도           6.85        17.85        -3.25   
            충청남도         -1.90        13.90       -20.80   
            충청북도         -3.10        13.30       -19.60   

                   월합강수량(00~24h만)(mm)_1월  합계 일사량(MJ/m2)_1월  평균기온(°C)_2월  \
2000 spring 강원도                    48.10           152.990         -3.3   
            경기도                    52.20           202.280         -2.3   
            경상남도                   21.90           173.530          1.2   
            경상북도                   29.20           167.555          0.2   
            광주광역시                  33.70           176.490          0.6   
...                                  ...               ...          ...   
2021 spring 전라남도                   33.30           275.670          5.2   
            전라북도                   36.00           274.140          3.7   
            제주도                    82.85           218.455          9.2   
            충청남도                   26.95           255.445          2.9   
            충청북도                   18.20           285.645          1.8   

                   최고기온(°C)_2월  최저기온(°C)_2월  월합강수량(00~24h만)(mm)_2월  \
2000 spring 강원도           7.70       -15.80                   5.70   
            경기도           8.30       -15.05                   1.55   
            경상남도         13.50        -9.70                   0.00   
            경상북도         10.40       -12.90                   0.00   
            광주광역시        12.40        -7.60                  15.40   
...                        ...          ...                    ...   
2021 spring 전라남도         21.65        -5.40                  33.40   
            전라북도         21.90        -7.60                  33.00   
            제주도          22.95        -2.30                  52.00   
            충청남도         19.35       -10.20                  15.30   
            충청북도         22.10       -11.90                  12.30   

                   합계 일사량(MJ/m2)_2월  ...  평균기온(°C)_11월  최고기온(°C)_11월  \
2000 spring 강원도             329.070  ...          4.10         20.30   
            경기도             317.510  ...          4.70         20.75   
            경상남도            360.910  ...          7.90         21.60   
            경상북도            398.050  ...          6.40         20.70   
            광주광역시           366.470  ...          8.40         22.10   
...                             ...  ...           ...           ...   
2021 spring 전라남도            325.740  ...          9.95         21.95   
            전라북도            337.560  ...          8.90         22.60   
            제주도             309.905  ...         14.05         22.50   
            충청남도            343.280  ...          7.95         21.85   
            충청북도            359.560  ...          6.40         19.90   

                   최저기온(°C)_11월  월합강수량(00~24h만)(mm)_11월  합계 일사량(MJ/m2)_11월  \
2000 spring 강원도           -8.40                   30.20            231.670   
            경기도           -7.90                   25.00            262.150   
            경상남도          -4.20                   56.00            301.480   
            경상북도          -6.90                   51.50            229.380   
            광주광역시         -3.00                   53.80            295.020   
...                         ...                     ...                ...   
2021 spring 전라남도          -1.45                   51.95            302.590   
            전라북도          -2.80                   86.40            